 <div style="text-align:center;">
  <span style="color:green; font-size:2em; font-weight:bold;">Visualisation</span><br><br>

Dans cette partie nous allons nous interesser à la visualisation de nos données. Pour les taux de chomage et de croissance, nous allons se concentrer uniquement sur les observations de la période s'etallant de **2019** à **2023**. Nous voulons à partir de cela visualiser les niveaux moyens de chomage et de croissance économique pour les cinq dernières années.
L'objectif ici est de mettre en pratique les méthodes vues en cours.

# <span style="color:green">I. Taux de chômage à travers le monde</span><br><br> 

In [ ]:
import declarations as d 

In [ ]:
# Importation de la DF construite initialement dans le fichier "Préparation_données_copy.ipynb"
final_data= d.pd.read_csv('C:/Projet-Python/monmodule/bases/final_data.csv', index_col=0)

In [ ]:
# Visualisation
final_data.head()

Nous allons à ce stade tronquer la Dataframe df_Unemployement, pour ne garder que les années à partir de 2019.

In [ ]:
# Tronquer la DataFrame pour garder uniquement les lignes à partir de 2019
df_tronque = final_data.loc[final_data.index >= '2019']

# Afficher la DataFrame tronquée
df_tronque.head()

Nous calculons la moyenne des taux de chômage sur la période considérée (2019-2023)

In [ ]:
# Calculer la moyenne des taux de chomage sur la période considérée
unemployment_mean =  df_tronque.groupby('COUNTRY')['Unemployment_rate'].mean().reset_index()

In [ ]:
# Visualisation
unemployment_mean.head(50)

## <span style="color:green">I.1. Carte statique </span><br><br> 

## <span style="color:green">I.1. Carte interactive : cartes choroplèthes  </span><br><br>  

Une carte choroplèthe est une carte composée de polygones colorés. Elle est utilisée pour représenter les variations spatiales d’une quantité.
Précisemment, nous optons ici pour la carte choroplèthe **px.choropleth** de la bibliothèque **Plotly Express**, cette dernière étant plus adapté aux notebooks jupyter.

Pour obtenir des données spatiale sur les pays de nos Dataframes, nous allons recourir au jeu de données **"naturalearth_lowres"** fourni par GeoPandas.

Les méthodes utilisées ici sont essentiellement ceux des bibliothèques **geopandas** et **plotly**.

In [ ]:
# Import des packages 
import declarations as d

In [ ]:
import plotly.express as px
import geopandas as gpd
# Recours au jeu de données "naturalearth_lowres"
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
# Visualisation
world.head(20)

Nous nous intéressons essentiellement aux colonnes suivantes du jeu de données "naturalearth_lowres" (la Dataframe "World") : 
     <div style="margin-left: 20px;">
    -name : Nom des pays. <br>
    -iso_a3 : Code ISO 3 des pays. <br>
    -geometry : Colonne géométrique contenant les frontières des pays
    </div>

Ensuite nous procédons à une jointure entre la DF "World" et la DF "unemployment_mean". La jointure se fait sur la colonne "iso_a3" de la DF "World" et la colonne "Country" de la DF "unemployment_mean".

In [ ]:
# Jointure
merged = gpd.GeoDataFrame(d.pd.merge(world, unemployment_mean, left_on='iso_a3', right_on='COUNTRY', how='left'))

In [ ]:
# Visualisation de la jointure
merged.tail(100)

In [ ]:
# Créer une carte centrée sur la position moyenne des pays
map_center = [merged['geometry'].centroid.y.mean(), merged['geometry'].centroid.x.mean()]

# Utiliser Plotly Express pour créer la carte choroplèthe
fig = px.choropleth(
    merged,
    geojson=merged.geometry,
    locations=merged.index,
    color='Unemployment_rate',
    color_continuous_scale="Viridis",
    range_color=(merged['Unemployment_rate'].min(), merged['Unemployment_rate'].max()),
    labels={'Unemployment mean': 'Unemployment Rate (%)', 'name': 'Country'},
    title='Unemployment Rate by Country',
    hover_name='name'
)

# Ajouter une couche choroplèthe à la carte
fig.update_geos(fitbounds="locations", visible=True)

fig.update_layout ( margin = { "r" : 0 , "t" : 0 , "l" : 0 , "b" : 0 }) 

# Afficher la carte interactive
fig.show()

## A VOIR 

In [ ]:
import folium

# Créer une carte folium centrée sur la position moyenne des pays
map_center = [merged['geometry'].centroid.y.mean(), merged['geometry'].centroid.x.mean()]
mymap = folium.Map(location=map_center, zoom_start=2)

# Ajouter une couche choroplèthe avec les taux de chômage
folium.Choropleth(
    geo_data=merged,
    name='choropleth',
    data=merged,
    columns=["name", 'Unemployment_rate'],
    key_on='feature.properties.iso_a3',
    fill_color="BuPu",
    fill_opacity=0.2,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)'
).add_to(mymap)

# Ajouter des popups pour afficher les taux de chômage
for idx, row in merged.iterrows():
    # Add Marker only if Unemployment rate is not NaN
    if not d.pd.isna(row['Unemployment_rate']):
        folium.Marker(
            location=[row['geometry'].centroid.y, row['geometry'].centroid.x],
            popup=f"{row['name']}<br>Unemployment Rate: {row['Unemployment_rate']}",
            icon=folium.Icon(icon='star', color='green')
        ).add_to(mymap)

# Afficher la carte interactive
mymap

## <span style="color:green">I.2. Séries temporelles  </span><br><br>

In [ ]:
# Visualisation
final_data.head(5)

In [ ]:
import pandas as pd
import plotly.express as px

# S'assurer que l'index est de type date
final_data.index = pd.to_datetime(final_data.index)

# Créer un graphe interactif pour les séries temporelles avec Plotly Express
fig = px.line(final_data.Unemployment_rate, x=final_data.index, y=final_data.Unemployment_rate, color=final_data.COUNTRY,
              labels={'index': 'Date', 'Unemployment_rate': 'Unemployment Rate'},
              title='Série temporelle du taux de chômage par pays',
              template='plotly_dark')

# Afficher le plot interactif
fig.show()


# <span style="color:green">II. Croissance économique à travers le monde</span><br><br> 